# 컴포넌트 스타일링

<br>

<hr>

<br>

## 01. 전통적인 방법의 스타일링

<br>

### 인라인 스타일
- JSX요소의 `style` 속성에 스타일 객체를 지정

```jsx
export default function App() {
    const styles = {
        backgroundColor: 'blue',
        color:'white',
        fontSize: '16px',
        padding: '10px'
    };

    return (
        <>
            <h1 style={styles}>Inline Style</h1>
        </>
    )
}
```

<br>

### 글로벌 스타일
- **css 확장자를 가진 파일에 CSS코드를 작성하고, 이를 컴포넌트에서 로드**
- **CSS파일을 한번 로드하면 해당 스타일은 전체 애플리케이션에 전역으로 적용됨**
  
  $\rightarrow$ 글로벌 스타일은 `main.tsx`파일에서 임포트 하는 경우가 많음

<br>

- `App.css`

```css
.btn {
  width: 100px;
  height: 50px;
  border: none;
  background-color: gray;
  color: white;
  font-size: 14px;
  border-radius: 10px;
  margin: 0 10px;
}
```

<br>

- `src/components/Child.tsx`

```tsx
export default function Child() {
    return <button className='btn'>Child Button</button>
}
```

<br>

- `App.tsx`

```tsx
import './App.css';
import Child from './components/Child';

export default function App() {
    return (
        <>
            <button className='btn'>App Button</button>
            <Child />
        </>
    )
}

```

<br>

### CSS 모듈
- **CSS모듈은 파일 확장자가 `.module.css`로 끝나는 파일에 스타일을 작성한 뒤 이를 컴포넌트에서 불러와 사용하는 방식**
- **CSS모듈은 스타일이 로컬 스코프(Local Scope)를 가짐** 
  
  $\rightarrow$ **특정 컴포넌트에만 적용되며, 스타일을 독립적으로 관리할 수 있음**
- 클래스 이름이 고유한 이름으로 자동 변환되기 때문에, 다른 컴포넌트의 클래스 이름이 중복되는 문제도 예방 가능

<br>

- `App.module.css`

```css
.btn {
  width: 100px;
  height: 50px;
  border: none;
  background-color: gray;
  color: white;
  font-size: 14px;
  border-radius: 10px;
  margin: 0 10px;
}
```

<br>

- `App.tsx`

```tsx
import styles from "./App.module.css";
import Child from "./components/Child";

export default function App() {
  return (
    <>
      <button className={styles.btn}>App Button</button>
      <Child />
    </>
  );
}
```

<br>

- `src/components/Child.tsx`

```tsx
export default function Child() {
    return <button className='btn'>Child Button</button>
}
```

<br>

- `App` 컴포넌트의 버튼에만 스타일이 적용되고, `Child` 컴포넌트의 버튼은 영향을 받지 않음

<img src='img/0501.png' width=300>

<br>

### `classnames` 라이브러리
- 글로벌 스타일이나 CSS 모듈을 사용할 떄 상황에 따라 클래스 이름을 조건부로 적용하는 경우 
  
  $\rightarrow$ `classnames` 라이브러리
  
  $\rightarrow$ **리액트를 포함한 자바스크립트 프레임워크에서 CSS 클래스 이름을 동적으로 조합하고, 관리**

<br>

- 설치
  - 설치 완료 후, `package.json` 파일의 `dependencies` 항목에 `classnames`가 추가
  
```bash
$ npm i classnames
```

<br>

#### `classNames()`
- https://www.npmjs.com/package/classnames
- **전달된 클래스 이름을 공백으로 구분한 하나의 문자열로 합쳐줌**
  - **`false`, `null`, `undefined`, 0, `''` 등 불필요한 값은 자동으로 무시**

```tsx
const btnClass = classNames('btn', 'primary');
console.log(btnClass) // btn primary
```

<br>

#### 글로벌 스타일에서 사용
- `classnames`라이브러리는 글로벌 스타일 방식에서 매우 유용하게 활용

<br>

- `App.css`

```css
.btn {
  width: 100px;
  height: 50px;
  border: none;
  background-color: gray;
  color: white;
  font-size: 14px;
  border-radius: 10px;
  margin: 0 10px;
}
.is-active {
  background-color: blue;
}
```

<br>

- `App.tsx`
  - **`classNames.bind()` : CSS 모듈 객체 (`styles`)와 연결**

```tsx
import styles from "./App.module.css";
import classNames from "classnames/bind";
import Child from "./components/Child";

export default function App() {
  const isActive = true;
  const cx = classNames.bind(styles);
  return (
    <>
      <button className={cx({ btn: true, "is-active": isActive })}>
        App Button
      </button>
      <Child />
    </>
  );
}
```

<br>

<hr>

<br>


## CSS-in-JS 스타일링
- 관심사 분리 (Separation of Concern) 소프트웨어 설계 원칙 : HTML, CSS, JS를 각각 분리해 개발하는 구조
  $\rightarrow$ 웹 애플리케이션이 더 복잡해짐에 따라 문제점 발생

1. **전역 스코프 문제** : 클래스 이름 충돌이 빈번하게 발생하며, 이를 방지하기 위한 명명 규칙 관리도 복잡
2. **상태 기반 스타일링의 어려움** : UI의 상태는 JS로, 스타일은 CSS에 정의하므로, 상태 변화에 따른 스타일 조작이 번거롭고 복잡
3. **유지보수의 어려움** : 프로젝트 규모가 커질수록, 어떤 스타일이 어떤 컴포넌트에 적용되는지 파악하기 힘들고, 사용하지 않는 CSS 객체를 제거하는 것이 어려움

$\rightarrow$ **CSS-in-JS** : **자바스크립트 파일 안에 스타일을 정의하고 적용하는 방식**

<br>

### `styled-components`
- JS코드 안에 스타일이 적용된 컴포넌트를 생성하는 방식
- `styled` 객체는 HTML 태그 이름에 해당하는 함수를 제공해, 이 함수를 사용하면 스타일 컴포넌트를 생성
  
<br>

```bash
$ npm install styled-components
```

<br>

#### Tagged Template Literal
- 템플릿 리터럴 앞에 함수를 붙여 문자열을 처리할 수 있는 자바스크립트 문법
- 함수는 템플릿 리터럴의 문자열 부분과 `${}`로 감싼 표현식을 분리된 배열 형태로 전달받아, 이를 바탕으로 원하는 방식으로 문자열을 가공할 수 있음

```tsx
function tagFunction(strings, ...values) {
  console.log(strings);
  console.log(values);
}
const name ='John';
const age = 25;
tagFunction`Hello, my name is ${name} and I am ${age} years old.`;

// 출력 결과
// ['Hello, my name is', 'and I am', ' years old.']
// ['John', 25]
```

<br>

#### 기본 사용법

```tsx
import styled, { css } from "styled-components";

const Button = styled.button<{ $primary?: boolean }>`
  background: transparent;
  border-radius: 3px;
  border: 2px solid #bf4f74;
  color: #bf4f74;
  margin: 0 1em;
  padding: 0.25em 1em;
  ${(props) =>
    props.$primary &&
    css`
      background: #bf4f74;
      color: white;
    `}
`;
export default function App() {
  return <Button $primary>Click Me</Button>;
}
```

<br>

#### `emotion`
- 널리 사용하고 유연한 CSS-in-JS 라이브러리
- https://emotion.sh/docs/introduction
  
<br>

```bash
$ npm i @emotion/css
```

<br>

#### 기본 사용법

```tsx
import { css } from "@emotion/css";

export default function App() {
  const isActive = false;
  return (
    <button
      className={css`
        background: ${isActive ? "#bf4f74" : "transparent"};
        border-radius: 3px;
        border: 2px solid #bf4f74;
        color: ${isActive ? "white" : "#bf4f74"};
        margin: 0 1em;
        padding: 0.25em 1em;
      `}
    >
      Click Me
    </button>
  );
}
```

<br>

### `vanilla-exract`
- **제로 런타임 : 애플리케이션이 실행 될 때, 스타일을 생성하거나 적용하는 데 추가 비용이 발생하지 않음**
  - 작성한 스타일이 빌드 타임에 정적 CSS파일로 변환 $\rightarrow$ 실행 중 스타일을 생성하지 않으므로 런ㅌ나임 비용이 없음
  - 초기 렌더링 속도가 빠르며, CSS 파일이 정적이므로 웹 브라우저 캐싱이 가능

<br>

```bash
$ npm install @vanilla-extract/css
$ npm install --save-dev @vanilla-extract/vite-plugin
```

<br>

#### 기본 사용법
- `css.ts`  확장자를 가진 파일에 CSS를 작성
- 각 `style()` 함수는 CSS 속성을 객체 형태로 작성하며, 반환된 클래스 이름이 변수에 저장됨

  $\rightarrow$ 이 변수들을 `className`에 할당하면 해당 스타일이 적용  

- `vanilla-extract`는 런타임에서 CSS를 생성하지 않지만, 조건에 따라 `className`을 선택적으로 할당하는 방식으로 동적 스타일을 적용

<br>

- `App.css.ts`
  - `button`은 항상 적용되고,  `active`클래스는 `AND`연산자를 사용해 조건이 `true`일 때만 추가

```ts
import { style } from "@vanilla-extract/css";

export const container = style({
  padding: "1rem",
});

export const button = style({
  background: "transparent",
  borderRadius: "3px",
  border: "2px solid #bf4f74",
  color: "#bf4f74",
  margin: "0 1em",
  padding: "0.25em 1em",
  ":hover": {
    backgroundColor: "#bf4f74",
    color: "white",
  },
});

export const active = style({
  backgroundColor: "blue",
  border: "2px solid blue",
  color: "white",
});
```

<br>

- `App.tsx`

```tsx
import { container, button, active } from "./App.css.ts";

export default function App() {
  const isActive = false;
  return (
    <div className={container}>
      <button className={`${button} ${isActive && active}`}>Click me</button>
    </div>
  );
}
```

<br>

<hr>

<br>

## 03. Tailwind
- https://tailwindcss.com/docs/installation/using-vite
- Tailwind CSS는 유틸리티 퍼스트 스타일링 철학을 따르는 프레임워크 중 가장 인기 잇는 라이브러리
  - **유틸리티 퍼스트** : 작고 재사용 가능한 클래스를 조합해 스타일을 만드는 방식 
- 각 유틸리티 클래스는 보통 하나의 CSS 속성과 일대일로 대응하며, 이 클래스들을 `class`속성에 나열해 원하는 스타일을 생성
  
  $\rightarrow$ CSS 파일 없이 HTML이나 JSX 코드 안에서 클래스만으로 스타일을 적용 가능

<br>

### 설치

```bash
$ npm install tailwind @tailwindcss/vite
```

#### **설치 후 `vite.config.ts`에 플러그인 추가**

```ts
import tailwindcss from '@tailwindcss/vite';

export default defineConfig({
  plugins: [tailwindcss()],
})
```

#### 라이브러리 로드
- `src` 폴더에 `index.css` 파일을 생성 후

```css
@import 'tailwindcss';
```

<br>

- 생성한 `index.css` 파일을 `main.tsx`에서 import

```tsx
import './index.css'; 
```

<br>

#### 사용
- Tailwind CSS는 유틸리티 클래스를 HTML요소에 직접 적용하여 설정하는 방식

```tsx
export default function App() {
  return (
    <>
      <button
        className="bg-transparent rounded-[3px] border-2 border-[#bf4f74] 
                        text-[#bf4f74] m-[0.1em] py-[0.25em] px-[1em]"
      >
        Click Me
      </button>
    </>
  );
}
```

#### CSS 속성
- https://tailwindcss.com/docs/installation/using-vite

| Tailwind CSS 클래스 | CSS 속성 |
| - | - |
| `bg-transparent` | `background-color:transparent;` |
| `rounded-[3px]` | `border-radius: 3px;` |
| `border-2` | `border-width: 2px;` |
| `border-[#bf4f74]` | `border-color: #bf4f74;` |
| `text-[#bf4f74]` | `text-color: #bf4f74;` |
| `m-[0.1em]` | `margin: 0.1em;` |
| `py-[0.25em]` | `padding-top: 0.25em;`<br>`padding-bottom: 0.25em;` |
| `px-[1em]` | `padding-left: 1em;`<br>`padding-right: 1em;` |

<br>

### `tailwind-merge`
- **`tailwind-merge`는 Tailwind CSS 클래스명을 충돌 없이 병합할 수 있게 해주는 유틸리티 라이브러리**
  - 동적으로 클래스명을 조합해야 할 때 유용
  
<br>

```bash
$ npm i tailwind-merge
```

<br>

- **`twMerge()`** : 여러 Tailwind CSS 클래스를 결합할 때 중복되거나 충돌되는 클래스가 있으면 마지막 클래스를 우선시해 중복을 제거

```tsx
import { twMerge } from "tailwind-merge";

export default function App() {
  return (
    <>
      <button
        className={twMerge(
          "bg-transparent text-[#bf4f74] rounded-[3px] border-2 border-[#bf4f74] m-[0.1em] py-[0.25em] px-[1em]",
          "bg-rose-500 text-white"
        )}
      >
        Click Me
      </button>
    </>
  );
}
```

<br>

- **`twJoin()`** : 여러 클래스를 단순히 문자열로 연결하며, 중복되거나 충돌되는 클래스도 그대로 유지

<br>

<hr>

<br>

## 04. 이미지 렌더링
- **컴포넌트에서 이미지를 렌더링하는 방식은 JSX문법을 사용**

<br>

### 이미지 리소스
- React 애플리케이션에서 이미지 파일은 보통 `public`과 `src` 폴더 중 하나에 추가 가능
- 이미지나 폰트 같은 리소스 파일은 일반적으로 `assets` 폴더에 저장

<br>

### 이미지 렌더링
- **`public` 폴더 하위에 저장한 이미지는 정적 (static) 리소스**

<br>

#### `img`태그 렌더링
- **파일의 경로는 절대 경로**

<br>

- `App.tsx`

```tsx
export default function App() {
  return (
    <>
      <img src='/assets/images/ocean.jpg' alt='ocean' />
    </>
  );
}
```

<br>

#### CSS 속성 렌더링

- `index.css`
  
```css
.bg {
    width: 640px;
    height: 360px;
    background: url('/assets/images/ocean.jpg') no-repeat center center;
}
```

<br>

- `App.tsx`

```tsx
export default function App() {
  return (
    <>
      <div className="bg"></div>
    </>
  );
}
```